# Validating the 10m Southern Africa Cropland Mask


## Description
Previously, in the `6_Accuracy_assessment_20m.ipynb` notebook, we were doing preliminary validations on 20m resolution testing crop-masks. The crop-mask was stored on disk as a geotiff. The final cropland extent mask, produced at 10m resolution, is stored in the datacube and requires a different method for validating.

> NOTE: A very big sandbox is required (256GiB RAM) to run this script. 

This notebook will output a `confusion error matrix` containing Overall, Producer's, and User's accuracy, along with the F1 score for each class.

***
## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

### Load Packages

In [1]:
import os
import sys
import glob
import rasterio
import datacube
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
import geopandas as gpd
from sklearn.metrics import f1_score
from rasterstats import zonal_stats

## Analysis Parameters

* `product` : name of crop-mask we're validating
* `bands`: the bands of the crop-mask we want to load and validate. Can one of either `'mask'` or `'filtered'`
* `grd_truth` : a shapefile containing crop/no-crop points to serve as the "ground-truth" dataset


In [2]:
product = "crop_mask_southern"
band = 'mask'
grd_truth = 'data/validation_samples.shp'




### Load the datasets

`the cropland extent mask`

In [3]:
#connect to the datacube
dc = datacube.Datacube(app='feature_layers')
    
#load 10m cropmask
ds = dc.load(product=product, measurements=[band], resolution=(-10,10)).squeeze()
print(ds)

<xarray.Dataset>
Dimensions:      (y: 211200, x: 220800)
Coordinates:
    time         datetime64[ns] 2019-07-02T11:59:59.999999
  * y            (y) float64 -2.112e+06 -2.112e+06 ... -4.224e+06 -4.224e+06
  * x            (x) float64 1.056e+06 1.056e+06 ... 3.264e+06 3.264e+06
    spatial_ref  int32 6933
Data variables:
    mask         (y, x) uint8 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0
Attributes:
    crs:           EPSG:6933
    grid_mapping:  spatial_ref


`Ground truth points`

In [4]:
#ground truth shapefile
ground_truth = gpd.read_file(grd_truth).to_crs('EPSG:6933')

# rename the class column to 'actual'
ground_truth = ground_truth.rename(columns={'Class':'Actual'})

# reclassifer into int
ground_truth['Actual'] = np.where(ground_truth['Actual']=='non-crop', 0, ground_truth['Actual'])
ground_truth['Actual'] = np.where(ground_truth['Actual']=='crop', 1, ground_truth['Actual'])
ground_truth.head()

,lon,lat,smpl_sampl,smpl_gfsad,Actual,ID,CODE,COUNTRY,geometry
0,15.201381,-26.335550,938,0,0,21,BOT,Botswana,POINT (1466724.714 -3245506.064)
1,28.722463,-29.657340,524,0,1,21,BOT,Botswana,POINT (2771323.653 -3620740.606)
2,21.767886,-32.657354,961,0,0,21,BOT,Botswana,POINT (2100302.362 -3949393.887)
3,27.354509,-30.097694,348,0,1,21,BOT,Botswana,POINT (2639334.810 -3669613.743)
4,20.365436,-33.198499,1178,0,0,21,BOT,Botswana,POINT (1964985.195 -4007569.023)



## Convert points into polygons

When the validation data was collected, 40x40m polygons were evaluated as either crop/non-crop rather than points, so we want to sample the raster using the same small polygons. We'll find the majority or 'mode' statistic within the polygon and use that to compare with the validation dataset.


In [5]:
#set radius (in metres) around points
radius = 20

#create circle buffer around points, then find envelope
ground_truth['geometry'] = ground_truth['geometry'].buffer(radius).envelope

### Calculate zonal statistics

We want to know what the majority pixel value is inside each validation polygon.

In [6]:
def custom_majority(x):
    a=np.ma.MaskedArray.count(x)
    b=np.sum(x)
    c=b/a
    if c>0.5:
        return 1
    if c<=0.5:
        return 0

In [7]:
#calculate stats
stats = zonal_stats(ground_truth.geometry,
                    ds[band].values,
                    affine=ds.geobox.affine,
                    add_stats={'majority':custom_majority},
                    nodata=255)

#append stats to grd truth df
ground_truth['Prediction']=[i['majority'] for i in stats]

ground_truth.head()

,lon,lat,smpl_sampl,smpl_gfsad,Actual,ID,CODE,COUNTRY,geometry,Prediction
0,15.201381,-26.335550,938,0,0,21,BOT,Botswana,"POLYGON ((1466704.714 -3245526.064, 1466744.71...",0
1,28.722463,-29.657340,524,0,1,21,BOT,Botswana,"POLYGON ((2771303.653 -3620760.606, 2771343.65...",1
2,21.767886,-32.657354,961,0,0,21,BOT,Botswana,"POLYGON ((2100282.362 -3949413.887, 2100322.36...",0
3,27.354509,-30.097694,348,0,1,21,BOT,Botswana,"POLYGON ((2639314.810 -3669633.743, 2639354.81...",1
4,20.365436,-33.198499,1178,0,0,21,BOT,Botswana,"POLYGON ((1964965.195 -4007589.023, 1965005.19...",0


***

## Create a confusion matrix

In [8]:
confusion_matrix = pd.crosstab(ground_truth['Actual'],
                               ground_truth['Prediction'],
                               rownames=['Actual'],
                               colnames=['Prediction'],
                               margins=True)

confusion_matrix

Prediction,0,1,All
Actual,,,
0,107,6,113
1,16,33,49
All,123,39,162


### Calculate User's and Producer's Accuracy

`Producer's Accuracy`

In [9]:
confusion_matrix["Producer's"] = [confusion_matrix.loc[0, 0] / confusion_matrix.loc[0, 'All'] * 100,
                              confusion_matrix.loc[1, 1] / confusion_matrix.loc[1, 'All'] * 100,
                              np.nan]

`User's Accuracy`

In [10]:
users_accuracy = pd.Series([confusion_matrix[0][0] / confusion_matrix[0]['All'] * 100,
                                confusion_matrix[1][1] / confusion_matrix[1]['All'] * 100]
                         ).rename("User's")

confusion_matrix = confusion_matrix.append(users_accuracy)

`Overall Accuracy`

In [11]:
confusion_matrix.loc["User's","Producer's"] = (confusion_matrix.loc[0, 0] + 
                                                confusion_matrix.loc[1, 1]) / confusion_matrix.loc['All', 'All'] * 100

`F1 Score`

The F1 score is the harmonic mean of the precision and recall, where an F1 score reaches its best value at 1 (perfect precision and recall), and is calculated as:

$$
\begin{aligned}
\text{Fscore} = 2 \times \frac{\text{UA} \times \text{PA}}{\text{UA} + \text{PA}}.
\end{aligned}
$$

Where UA = Users Accuracy, and PA = Producer's Accuracy

In [12]:
fscore = pd.Series([(2*(confusion_matrix.loc["User's", 0]*confusion_matrix.loc[0, "Producer's"]) / (confusion_matrix.loc["User's", 0]+confusion_matrix.loc[0, "Producer's"])) / 100,
                    f1_score(ground_truth['Actual'].astype(np.int8), ground_truth['Prediction'].astype(np.int8), average='binary')]
                         ).rename("F-score")

confusion_matrix = confusion_matrix.append(fscore)

### Tidy Confusion Matrix

* Limit decimal places,
* Add readable class names
* Remove non-sensical values 

In [13]:
# round numbers
confusion_matrix = confusion_matrix.round(decimals=2)

In [14]:
# rename booleans to class names
confusion_matrix = confusion_matrix.rename(columns={0:'Non-crop', 1:'Crop', 'All':'Total'},
                                            index={0:'Non-crop', 1:'Crop', 'All':'Total'})

In [15]:
#remove the nonsensical values in the table
confusion_matrix.loc["User's", 'Total'] = '--'
confusion_matrix.loc['Total', "Producer's"] = '--'
confusion_matrix.loc["F-score", 'Total'] = '--'
confusion_matrix.loc["F-score", "Producer's"] = '--'

In [16]:
confusion_matrix

,Non-crop,Crop,Total,Producer's
Actual,,,,
Non-crop,107.00,6.00,113.0,94.69
Crop,16.00,33.00,49.0,67.35
Total,123.00,39.00,162.0,--
User's,86.99,84.62,--,86.42
F-score,0.91,0.75,--,--


### Export csv

In [17]:
confusion_matrix.to_csv('results/Southern_10m_accuracy_assessment_confusion_matrix.csv')

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Last modified:** Dec 2020
